<a href="https://colab.research.google.com/github/ariana-lab/Prog_cientifica/blob/main/Copia_de_CI_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Estructura del proyecto

Primero vamos a generar la estructura básica de carpetas y archivos que usaremos como ejemplo. Esto es útil para tener algo concreto sobre lo que trabajar y probar el flujo CI.

Trabajamos lo siguiente:

* Creamos un pequeño proyecto Python.
* archivo main.py con una función simple.
* prueba unitaria (test_main.py) para asegurarnos de que esa función funciona.
* Dejamos listo el archivo donde irá nuestro workflow de CI.

Vamos a usar **pytest** para las pruebas

* pytest es un framework de testing que permite:

  * Escribir pruebas unitarias de manera simple y clara.

  * Ejecutar automáticamente pruebas para verificar que el código funciona.

  * Mostrar mensajes útiles cuando una prueba falla.

  * Extender el testing con fixtures, parametrización y plugins avanzados.

**Ejemplo de uso de pytest**

* Debes ejecutar pytest desde el terminal o en colab (dentro de tu proyecto)
* pytest busca automáticamente todos los archivos que comienzan con test_ o terminan en _test.py
* Dentro de esos archivos, ejecuta todas las funciones que comienzan con test_

pytest encuentra las pruebas automáticamente

In [5]:
import os # permite interactuar con el sistema de archivos

# Carpetas del proyecto
dirs = [
    "my_project/.github/workflows",  # Carpeta donde irá el workflow de GitHub Actions (aquí esta el archivo .yml)
    "my_project/src",                # Carpeta para el código fuente del proyecto
    "my_project/tests"              # Carpeta donde estarán las pruebas unitarias
]

# Archivos base del proyecto
files = {
    "my_project/requirements.txt": "pytest\nflake8\n",  # Dependencias del proyecto para ejecutar pruebas
    "my_project/src/main.py": "def add(a, b):\n    return a + b\n",  # Función simple a testear
    "my_project/tests/test_main.py": "from src.main import add\n\ndef test_add():\n    assert add(2, 3) == 5\n",  # Prueba unitaria
    "my_project/.github/workflows/ci.yml": "",  # Aquí luego pondremos el workflow CI
    "my_project/README.md": "# Proyecto de ejemplo para CI\n"  # Documentación inicial
}

# Crear directorios y archivos
for d in dirs:
    os.makedirs(d, exist_ok=True) # el argumento evita errores si la carpeta ya existe

for path, content in files.items():
    with open(path, "w") as f:
        f.write(content)

print("Estructura creada")

Estructura creada


# 2. workflow de GitHub Actions

Escribimos el archivo .github/workflows/ci.yml, que es donde GitHub Actions define qué hacer cada vez que alguien sube código (push) o crea un pull request.

¿Qué hace este archivo?

* Activa el flujo de CI al hacer push o pull_request en la rama main.

* Corre el código en Linux, Windows y macOS.

* Usa varias versiones de Python.

* Instala las dependencias del proyecto.

* Revisa el estilo del código con flake8.

* Corre las pruebas unitarias con pytest.



In [3]:
ci_workflow = """
name: CI Pipeline

on:
  push:
    branches: [main]  # Ejecuta el CI cuando hay un push a la rama main
  pull_request:
    branches: [main]  # También en los PR hacia la rama main

jobs:
  test:
    name: Run Tests
    runs-on: ${{ matrix.os }}  # Corre en múltiples sistemas operativos

    strategy:
      matrix:
        os: [ubuntu-latest, windows-latest, macos-latest]  # Soporte multiplataforma
        python-version: [3.9, 3.10, 3.11]                   # Pruebas con varias versiones de Python

    steps:
    - name: Checkout code
      uses: actions/checkout@v4  # Clona el repo

    - name: Set up Python
      uses: actions/setup-python@v5
      with:
        python-version: ${{ matrix.python-version }}

    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt

    - name: Run linter
      run: |
        flake8 src tests  # Linter para verificar estilo de código

    - name: Run tests
      run: |
        pytest  # Corre las pruebas unitarias
"""

# Guardar el workflow en el archivo correspondiente
with open("my_project/.github/workflows/ci.yml", "w") as f:
    f.write(ci_workflow.strip())

print("Workflow CI creado")

FileNotFoundError: [Errno 2] No such file or directory: 'my_project/.github/workflows/ci.yml'

# 3. Probar el proyecto localmente

En windows

cd my_project

python -m venv venv

venv\Scripts\activate

pip install -r requirements.txt

pytest


# 4. Subir a GitHub directamente desde Google Colab

Requisitos antes de comenzar

Ve a https://github.com/settings/tokens.

Crea un token de acceso personal (classic) con permisos:

* repo

* workflow

Guarda ese token (lo usaremos temporalmente desde Colab).

**Instalación de GitPython y configuración de Git**

In [6]:
!pip install GitPython > /dev/null
import git
import os

# Cambiar al directorio del proyecto
os.chdir("/content/my_project")

# Configura tu identidad de Git
!git config --global user.email " "
!git config --global user.name " "

**Inicializar repo y hacer el push**

In [ ]:
import os
import subprocess #Permite ejecutar comandos del sistema como si estuvieras en la terminal.
from getpass import getpass #Permite pedir datos al usuario sin que se muestren en pantalla (ideal para contraseñas o tokens)

# 🔐 Ingresar el token de forma segura
token = getpass("🔐 Ingresa tu GitHub token: ")
usuario = "ediomardones"
nombre_repo = "python_test"

# 📁 Ir al proyecto
os.chdir("/content/my_project")

# 🧹 Limpiar .git anterior si existía
!rm -rf .git

# 🔧 Inicializar y configurar Git
!git init #Crea un nuevo repositorio Git en la carpeta actual.
!git config user.name "ediomardones" #Configura el nombre y correo que aparecerán en el commit.
!git config user.email "emardones.arias@gmail.com"
!git add . #Agrega todos los archivos del proyecto al área de staging de Git.
!git commit -m "Primer commit desde Colab con CI" #Crea un snapshot de los archivos actuales con un mensaje descriptivo.
!git branch -m main #estándar actual en github

# 🔗 Crear el remote con token incluido

#Crea una URL que incluye el token para autenticación sin necesidad de
# ingresar usuario y contraseña manualmente.
remote_url = f"https://{token}@github.com/{usuario}/{nombre_repo}.git"
# Asocia el repositorio local a ese remoto, con el nombre origin.
!git remote add origin {remote_url}

# 🚀 Push con control de errores
#El comando git push -u origin main sube el código al repositorio remoto en la rama main
try:
    subprocess.run(["git", "push", "-u", "origin", "main"], check=True)
    print("✅ ¡Proyecto subido correctamente desde Colab!")
except subprocess.CalledProcessError as e:
    print("❌ Error al hacer push:")
    print("STDOUT:\n", e.stdout)
    print("STDERR:\n", e.stderr)

# Es compatible con integración continua (CI) porque el código se puede enlazar a un workflow.


🔐 Ingresa tu GitHub token: ··········
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/my_project/.git/
[master (root-commit) 510e3cc] Primer commit desde Colab con CI
 5 files changed, 48 insertions(+)
 create mode 100644 .github/workflows/ci.yml
 create mode 100644 README.md
 create mode 100644 requirements.txt
 create mode 100644 src/main.py
 create mode 100644 tests/test_main.py
✅ ¡Proyecto subido correctamente desde Colab!


**Ver tu Workflow de GitHub Actions paso a paso**

1. Ve a tu repositorio en GitHub:
👉 https://github.com/ediomardones/Prog-cientifica

2. Haz clic en la pestaña Actions (justo entre Pull requests y Projects).

3. Deberías ver una lista con algo como:
✔️ CI Pipeline – "push to main" – Passed o Running

4. Haz clic en la ejecución más reciente (la de hoy) para entrar a los detalles.

🔎 **Dentro de la ejecución verás:**

Jobs (por sistema operativo):

* Linux, Windows, macOS (dependiendo de tu matriz).

Cada uno con el estado:  passed |  failed |  running

**Dentro de cada Job:**

1. Set up job: GitHub reserva el runner.

2. Checkout code: Clona tu repo.

3. Set up Python: Instala la versión de Python del job.

4. Install dependencies: pip install.

5. Run linter: flake8 escanea tu código.

6. Run tests: corre pytest.

**¿Qué pasa si algo falla?**

* Verás un ❌ rojo en el paso que falló.

* Puedes hacer clic en ese paso y GitHub mostrará el log completo del error.

* Ejemplo: si flake8 falla, verás los archivos, líneas y errores de estilo detectados.

**¿Qué pasa si todo va bien?**

Verás un ✅ verde y el mensaje:

This workflow ran successfully.

¡Tu flujo CI está funcionando como se espera!

# Actividad

La función procesar_pago depende de una función externa verificar_saldo_en_banco que representa una llamada a un sistema de banco (por ejemplo, una API).

En esta actividad, no se debe ejecutar verificar_saldo_en_banco realmente, sino simular su comportamiento con mock.

Código entregado

In [ ]:
# src/pagos.py

from typing import Callable

def verificar_saldo_en_banco(usuario: str) -> float:
    """
    Simula una consulta a un sistema externo (no se implementa).
    """
    raise NotImplementedError("Esta función simula un sistema externo.")

def procesar_pago(usuario: str, monto: float, verificador: Callable[[str], float]) -> bool:
    """
    Retorna True si el usuario tiene saldo suficiente y se puede procesar el pago.
    """
    saldo = verificador(usuario)
    return saldo >= monto


**Parte 1: Crear las pruebas con mock**

Crea el archivo tests/test_pagos.py.

Usa pytest y unittest.mock para simular diferentes saldos disponibles.

In [8]:
!pip install pytest pytest-mock


In [10]:
# Simulación del archivo src/pagos.py

from typing import Callable

def verificar_saldo_en_banco(usuario: str) -> float:
    """
    Simula una consulta a un sistema externo (no se implementa).
    """
    raise NotImplementedError("Esta función simula un sistema externo.")

def procesar_pago(usuario: str, monto: float, verificador: Callable[[str], float]) -> bool:
    """
    Retorna True si el usuario tiene saldo suficiente y se puede procesar el pago.
    """
    saldo = verificador(usuario)
    return saldo >= monto



In [11]:
%%writefile test_pagos.py
from unittest.mock import Mock
from pagos import procesar_pago

def test_pago_con_saldo_suficiente():
    mock_verificador = Mock(return_value=100.0)
    assert procesar_pago("usuario1", 50.0, mock_verificador) is True

def test_pago_con_saldo_exacto():
    mock_verificador = Mock(return_value=50.0)
    assert procesar_pago("usuario2", 50.0, mock_verificador) is True

def test_pago_con_saldo_insuficiente():
    mock_verificador = Mock(return_value=30.0)
    assert procesar_pago("usuario3", 50.0, mock_verificador) is False


Writing test_pagos.py


In [12]:
%%writefile pagos.py
from typing import Callable

def verificar_saldo_en_banco(usuario: str) -> float:
    raise NotImplementedError("Esta función simula un sistema externo.")

def procesar_pago(usuario: str, monto: float, verificador: Callable[[str], float]) -> bool:
    saldo = verificador(usuario)
    return saldo >= monto


Writing pagos.py


In [13]:
!pytest test_pagos.py


============================= test session starts ==============================
platform linux -- Python 3.11.12, pytest-8.3.5, pluggy-1.5.0
rootdir: /content/my_project
plugins: mock-3.14.0, typeguard-4.4.2, langsmith-0.3.28, anyio-4.9.0
collected 3 items                                                              

test_pagos.py ...                                                        [100%]

============================== 3 passed in 0.02s ===============================


**Parte 2: Flujo CI con GitHub Actions**

Crea el flujo de trabajo en yaml

**Parte 3: Carga a tu repo de github**

**Parte 4: Ejecuta pruebas y haz seguimiento en github**